## **Analyze Voice of Social - Community**

In this notebook we retrieve and perform sentiment analysis on **L'Occitane en Provence Communities Tweets** using **TestBlob and Vader Python Libraries**.

To collect data, we will use Twitter API and get **Tweets containing specific hashtags** from the two brands. 

Geeksforgeeks Data Science tutorial: https://www.geeksforgeeks.org/extracting-tweets-containing-a-particular-hashtag-using-python/

## **Introduction: Libraries and Credentials** 

In [ ]:
# Import all needed libraries
import tweepy                   # Python wrapper around Twitter API
from google.colab import drive  # to mount Drive to Colab notebook

import pandas as pd
pd.set_option('display.max_colwidth', None) #to see more text
import json 
import csv
from datetime import date
from datetime import datetime
import time
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()

from textblob import TextBlob
from wordcloud import WordCloud
import string
import itertools
from collections import Counter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/path/to/data'

Mounted at /content/gdrive


In [ ]:
# Twitter API credentials
api_key = "XXXXXXXXXXXXXXXX"
api_secret_key = "XXXXXXXXXXXXXXXX"
access_token = "XXXXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXXXX"

In [ ]:
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# Helper function to save data into a JSON file
# file_name: the file name of the data on Google Drive
# file_content: the data you want to save
def save_json(file_name, file_content):
  with open(path + file_name, 'w', encoding='utf-8') as f:
    json.dump(file_content, f, ensure_ascii=False, indent=4)

In [ ]:
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
  while True:
    try:
      yield cursor.next()
    # Catch Twitter API rate limit exception and wait for 15 minutes
    except tweepy.RateLimitError:
      print("\nData points in list = {}".format(len(list_name)))
      print('Hit Twitter API rate limit.')
      for i in range(3, 0, -1):
        print("Wait for {} mins.".format(i * 5))
        time.sleep(5 * 60)
    # Catch any other Twitter API exceptions
    except tweepy.error.TweepError:
      print('\nCaught TweepError exception' )

In [ ]:
# Define Microsoft Translation function 

# Microsoft Translation
import requests, uuid, json

def microsoft_translate(text):
  # Add your subscription key and endpoint
  subscription_key = "XXXXXXXXXXXXXXXX"
  endpoint = "https://api.cognitive.microsofttranslator.com"

  # Add your location, also known as region. The default is global.
  # This is required if using a Cognitive Services resource.
  location = "westeurope"

  path = '/translate'
  constructed_url = endpoint + path

  params = {
      'api-version': '3.0',
      'to': 'en'
  }

  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(uuid.uuid4())
  }

  # You can pass more than one object in body.
  body = [{
      'text': str(text)
  }]

  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json()

  # print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

  return response[0]['translations'][0]['text']

## **Get the word tweets**

In [ ]:
# Python Script to Extract tweets of a
# particular Hashtag using Tweepy and Pandas
	
words = ["L'Occitane","L'Occitane en Provence","loccitane","loccitaneenprovence","LOCCITANE_UK","loccitane_jp"]
# Creating DataFrame using pandas
dftweets = pd.DataFrame(columns=['word','username', 'description', 'location', 'following',
							'followers', 'totaltweets', 'retweetcount', 'date', 'text', 'hashtags'])

for word in words:
  # Creating DataFrame using pandas
  db = pd.DataFrame(columns=['word','username', 'description', 'location', 'following',
							'followers', 'totaltweets', 'retweetcount', 'date', 'text', 'hashtags'])
  # initialize a list to hold all the Tweets
  alltweets = []
  # make initial request for most recent tweets 
  # (200 is the maximum allowed count)
  new_tweets = tweepy.Cursor(api.search, q=word,
				    since='2015-01-01', count=200, tweet_mode='extended').items()
  list_tweets = [tweet for tweet in new_tweets]
  # save most recent tweets
  alltweets.extend(list_tweets)
  # save the id of the oldest tweet less one to avoid duplication
  oldest = alltweets[-1].id - 1
  # keep grabbing tweets until there are no tweets left
  while len(list_tweets) > 0:
      print("getting tweets before %s" % (oldest))
      # all subsequent requests use the max_id param to prevent
      # duplicates
      new_tweets = tweepy.Cursor(api.search, q=word,
				    since='2015-01-01', count=200, tweet_mode='extended',max_id=oldest).items()
      list_tweets = [tweet for tweet in new_tweets]
      # save most recent tweets
      alltweets.extend(list_tweets)
      # update the id of the oldest tweet less one
      oldest = alltweets[-1].id - 1
      print("...%s tweets downloaded so far" % (len(alltweets)))
      ### END OF WHILE LOOP ###

  for tweet in alltweets:
    username = tweet.user.screen_name
    description = tweet.user.description
    location = tweet.user.location
    following = tweet.user.friends_count
    followers = tweet.user.followers_count
    totaltweets = tweet.user.statuses_count
    retweetcount = tweet.retweet_count
    hashtags = tweet.entities['hashtags']
    date = tweet.created_at

    # Retweets can be distinguished by a retweeted_status attribute,
	  # in case it is an invalid reference, except block will be executed
    try:
      text = tweet.retweeted_status.full_text
    except AttributeError:
      text = tweet.full_text
    hashtext = list()
    for j in range(0, len(hashtags)):
      hashtext.append(hashtags[j]['text'])
   
    # Here we are appending all the extracted information in the DataFrame
    ith_tweet = [word, username, description, location, following,
					followers, totaltweets, retweetcount, date, text, hashtext]
    db.loc[len(db)] = ith_tweet

  dftweets = dftweets.append(db)
  print(dftweets["word"].value_counts())

getting tweets before 1532356251279302657
...1658 tweets downloaded so far
L'Occitane    1658
Name: word, dtype: int64
getting tweets before 1532519355828670468
...18 tweets downloaded so far
L'Occitane                1658
L'Occitane en Provence      18
Name: word, dtype: int64
getting tweets before 1532358616535416831
...1905 tweets downloaded so far
loccitane                 1905
L'Occitane                1658
L'Occitane en Provence      18
Name: word, dtype: int64


IndexError: ignored

In [ ]:
# Get info about the data
dftweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3581 entries, 0 to 1904
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   word          3581 non-null   object        
 1   username      3581 non-null   object        
 2   description   3581 non-null   object        
 3   location      3581 non-null   object        
 4   following     3581 non-null   object        
 5   followers     3581 non-null   object        
 6   totaltweets   3581 non-null   object        
 7   retweetcount  3581 non-null   object        
 8   date          3581 non-null   datetime64[ns]
 9   text          3581 non-null   object        
 10  hashtags      3581 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 335.7+ KB


In [ ]:
# Dedup data
dftweets[dftweets['text'].duplicated()==False].shape

(2166, 11)

In [ ]:
# Dedup data
dftweets = dftweets[dftweets['text'].duplicated()==False]
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags
0,L'Occitane,RecOnlineShop,"こんにちは。AmazonやKindleストアでお買い物が好きでこれまで3,000,000円以上使ってきました…😅 そんな私が、気になるモノをご紹介します😄 ステマ反対なので、怪しいものはお金を貰っても紹介しません！",札幌,2852,2838,11158,0,2022-06-11 08:33:37,ロクシタンっていうと女性が使ってるかと思いますが、このサイズって男性にも邪魔にならなくていいかも！\n\nhttps://t.co/6BU0QR0xi9\nロクシタン(L'OCCITANE) フラワーミニハンドクリーム トリオ10ml,[]
1,L'Occitane,WaltherBeck,"En 1981, on craignait la venue des chars Soviétiques à Paris. Aujourd'hui je me surprends à l'espérer.🇷🇺🇲🇫",On avance et puis on voit,610,520,65459,5,2022-06-11 07:43:17,"🇷🇺🇨🇵 Après quelques semaines d'absence la #marque #française L'Occitane semble être de retour à Moscou mais avec une enseigne en cyrillique 🇷🇺 ""Л'Окситан"" https://t.co/Q2iIpIG3Yr","[marque, française]"
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,"@venusinfurrrs L'Occitan, L'Occitane en Provence",[]
4,L'Occitane,rachhparr,frannie’s mom (she is a dog),stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,@santanabandanas they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it. i need to get my ass to l’occitane,[]
5,L'Occitane,clubkano0855,#レシピ #料理 #グルメ #Amazon #アマゾン #相互フォロー #フォロバ100 #followmejp #無言フォローOK,,8768,8495,13682,0,2022-06-11 07:02:28,ロクシタン(L'OCCITANE) の ロクシタン(L'OCCITANE) チェリーブロッサム ソフトハンドクリーム 75ml を Amazon でチェック！ \nhttps://t.co/uIeGb4DWDK amazonより,[]


In [ ]:
dftweets.date.min()

Timestamp('2022-06-02 13:39:34')

In [ ]:
# How many tweets do we have by word? 
dftweets['word'].value_counts()

loccitane     1457
L'Occitane     709
Name: word, dtype: int64

In [ ]:
# Clean the text

# Create a function to clean the tweets

def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) #Revoming @mentions
  text = re.sub(r'#', '', text) # Removing the '#' simbol
  text = re.sub(r'RT[\s]+', '', text) # Removing RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Removing the hyper link
  text = text.lower() # make text lowercase
  text = re.sub('\[.*?\]', '', text) # removing text within brackets
  text = re.sub('\(.*?\)', '', text) # removing text within parentheses
  text = re.sub('\w*\d\w*', '', text) # removing numbers
  text = re.sub('\s+', ' ', text) # if there's more than 1 whitespace, then make it just 1
  text = re.sub('\n', ' ', text) # if there's a new line, then make it a whitespace
  text = re.sub('\"+', '', text) # removing any quotes
  text = re.sub('(\&amp\;)', '', text) # removing &amp;
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Get rid of all punctuation
  text = re.sub('(httptco)', '', text) # getting rid of `httptco`
  text = re.sub(r'[^\w\s]', '',text) # remove other punctuation

  return text

# Cleaning the text
dftweets['text'] = dftweets['text'].apply(cleanTxt)

# Show the cleaned text
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags
0,L'Occitane,RecOnlineShop,"こんにちは。AmazonやKindleストアでお買い物が好きでこれまで3,000,000円以上使ってきました…😅 そんな私が、気になるモノをご紹介します😄 ステマ反対なので、怪しいものはお金を貰っても紹介しません！",札幌,2852,2838,11158,0,2022-06-11 08:33:37,ロクシタンっていうと女性が使ってるかと思いますがこのサイズって男性にも邪魔にならなくていいかも ロクシタン フラワーミニハンドクリーム,[]
1,L'Occitane,WaltherBeck,"En 1981, on craignait la venue des chars Soviétiques à Paris. Aujourd'hui je me surprends à l'espérer.🇷🇺🇲🇫",On avance et puis on voit,610,520,65459,5,2022-06-11 07:43:17,après quelques semaines dabsence la marque française loccitane semble être de retour à moscou mais avec une enseigne en cyrillique локситан,"[marque, française]"
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane en provence,[]
4,L'Occitane,rachhparr,frannie’s mom (she is a dog),stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[]
5,L'Occitane,clubkano0855,#レシピ #料理 #グルメ #Amazon #アマゾン #相互フォロー #フォロバ100 #followmejp #無言フォローOK,,8768,8495,13682,0,2022-06-11 07:02:28,ロクシタン の ロクシタン チェリーブロッサム ソフトハンドクリーム を amazon でチェック amazonより,[]


## **Translate and Clean Tweets**

In [ ]:
# Check data shape
dftweets['text'].shape[0]

2166

In [ ]:
# Loop through Japanese Tweets for Translation
%%capture
for i in range(0,dftweets.shape[0]):
  try:
    print(i)
    dftweets['text'].iloc[i] = microsoft_translate(dftweets['text'].iloc[i])
    dftweets['description'].iloc[i] = microsoft_translate(dftweets['description'].iloc[i])
    dftweets['location'].iloc[i] = microsoft_translate(dftweets['location'].iloc[i])
  except: 
      print('error')

In [ ]:
# Clean the text Again

# Create a function to clean the tweets

def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) #Revoming @mentions
  text = re.sub(r'#', '', text) # Removing the '#' simbol
  text = re.sub(r'RT[\s]+', '', text) # Removing RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Removing the hyper link
  text = text.lower() # make text lowercase
  text = re.sub('\[.*?\]', '', text) # removing text within brackets
  text = re.sub('\(.*?\)', '', text) # removing text within parentheses
  text = re.sub('\w*\d\w*', '', text) # removing numbers
  text = re.sub('\s+', ' ', text) # if there's more than 1 whitespace, then make it just 1
  text = re.sub('\n', ' ', text) # if there's a new line, then make it a whitespace
  text = re.sub('\"+', '', text) # removing any quotes
  text = re.sub('(\&amp\;)', '', text) # removing &amp;
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Get rid of all punctuation
  text = re.sub('(httptco)', '', text) # getting rid of `httptco`
  text = re.sub(r'[^\w\s]', '',text) # remove other punctuation

  return text

# Cleaning the text
dftweets['text'] = dftweets['text'].apply(cleanTxt)
dftweets['description'] = dftweets['description'].apply(cleanTxt)
dftweets['location'] = dftweets['location'].apply(cleanTxt)

# Show the cleaned text
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[]
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]"
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[]
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[]
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[]


## **Sentiment Analysis**

### **Number of words and text length**

In [ ]:
dftweets['text_len'] = dftweets['text'].apply(lambda x: len(str(x).split()))
dftweets

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,text_len
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13
...,...,...,...,...,...,...,...,...,...,...,...,...
1898,loccitane,ShowDasMilhas,discover the world of miles simply and objectively,,32,82,372,0,2022-06-02 14:27:08,ciclic seguro celular rosa cha loccitane au bresil marisa no faro magazine luiza sephora petlove extra docpass assist card atlantica miles points tips tips,"[Milhas, Pontos, Dicas, Tips]",24
1899,loccitane,ShowDasMilhas,discover the world of miles simply and objectively,,32,82,372,0,2022-06-02 14:26:30,the best promotions da today monis dzarm editora tres loccitane en provence polishop riachuelo filóo americanas bebelier bobo carbee ciclic,[promocoes],20
1900,loccitane,0611_HONEY,super hero taim love cats and songs and nature and milk tea mom of cats is my treasure pokékara the goal is pokékara super sentai series tokusatsu series domination currently the song,,98,102,2994,0,2022-06-02 14:19:02,just vote for the activities you want to support,[1クリックで未来へのアクション],9
1901,loccitane,nekozoku,,tokyo nakameguro shibuya omotesando,270,99,8631,0,2022-06-02 14:18:05,just vote for the activities you want to support,[1クリックで未来へのアクション],9


In [ ]:
# Set number of words
dftweets.rename(columns={"text_len": "nb_words"}, inplace=True)
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13


In [ ]:
# Set Text len
dftweets['text_len'] = dftweets['text'].astype(str).apply(len)
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words,text_len
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22,116
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23,121
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4,31
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36,162
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13,85


### **Subjectivity and Polarity with TextBlob**

In [ ]:
# Create a function to get the subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

# Create two new columns
dftweets['Subjectivity'] = dftweets['text'].apply(getSubjectivity)
dftweets['Polarity'] = dftweets['text'].apply(getPolarity)

# Show the new dataframe with the new columns
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words,text_len,Subjectivity,Polarity
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22,116,0.800,-0.500000
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23,121,0.025,-0.053125
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4,31,0.000,0.000000
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36,162,1.000,0.250000
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13,85,0.350,0.100000


In [ ]:
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words,text_len,Subjectivity,Polarity
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22,116,0.800,-0.500000
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23,121,0.025,-0.053125
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4,31,0.000,0.000000
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36,162,1.000,0.250000
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13,85,0.350,0.100000


### **Polarity and Intensity with VADER**

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
sentiment = dftweets['text'].apply(lambda x: analyzer.polarity_scores(str(x)))
dftweets = pd.concat([dftweets,sentiment.apply(pd.Series)],1)

In [ ]:
dftweets.rename(columns={'Subjectivity':'subjectivity', 'Polarity':'polarity', 'neg':'negative',
                         'neu':'neutral', 'pos':'positive'}, inplace=True)
dftweets.head()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words,text_len,subjectivity,polarity,negative,neutral,positive,compound
0,L'Occitane,RecOnlineShop,hello i like shopping on amazon and kindle stores and have spent more than yen so far i will introduce things that i am interested in because i am against stemma i will not introduce suspicious things even if i get money,sapporo,2852,2838,11158,0,2022-06-11 08:33:37,i think that loccitane is used by women but this size may not be disturbing to men loccitane flower mini hand cream,[],22,116,0.800,-0.500000,0.000,0.708,0.292,0.8340
1,L'Occitane,WaltherBeck,in there were fears of soviet tanks coming to paris today i find myself hoping for it,we move forward and then we see,610,520,65459,5,2022-06-11 07:43:17,after a few weeks of absence the french brand loccitane seems to be back in moscow but with a sign in cyrillic локситан,"[marque, française]",23,121,0.025,-0.053125,0.000,1.000,0.000,0.0000
3,L'Occitane,frndvrgs,,,181,19,574,0,2022-06-11 07:05:29,loccitan loccitane in provence,[],4,31,0.000,0.000000,0.000,1.000,0.000,0.0000
4,L'Occitane,rachhparr,frannies mom,stuck in pittsburgh,355,165,3761,0,2022-06-11 07:05:03,they have a perfume that i once splurged on and wore for all of junior year and i have never felt so pretty than when i wore it i need to get my ass to loccitane,[],36,162,1.000,0.250000,0.096,0.792,0.112,0.1571
5,L'Occitane,clubkano0855,レシピ 料理 グルメ amazon アマゾン 相互フォロー followmejp 無言フォローok,,8768,8495,13682,0,2022-06-11 07:02:28,loccitanes loccitane cherry blossom soft hand ice cream checked on amazon from amazon,[],13,85,0.350,0.100000,0.000,0.602,0.398,0.6808


### **Save CSV File**

In [ ]:
dftweets.to_csv("/content/gdrive/MyDrive/The Role of Storytelling in Brand Communication/6. Voice of Community/Community_Tweet_Posts.csv")

In [ ]:
dftweets.tail()

,word,username,description,location,following,followers,totaltweets,retweetcount,date,text,hashtags,nb_words,text_len,subjectivity,polarity,negative,neutral,positive,compound
1898,loccitane,ShowDasMilhas,discover the world of miles simply and objectively,,32,82,372,0,2022-06-02 14:27:08,ciclic seguro celular rosa cha loccitane au bresil marisa no faro magazine luiza sephora petlove extra docpass assist card atlantica miles points tips tips,"[Milhas, Pontos, Dicas, Tips]",24,155,0.1,0.0,0.087,0.913,0.000,-0.2960
1899,loccitane,ShowDasMilhas,discover the world of miles simply and objectively,,32,82,372,0,2022-06-02 14:26:30,the best promotions da today monis dzarm editora tres loccitane en provence polishop riachuelo filóo americanas bebelier bobo carbee ciclic,[promocoes],20,140,0.3,1.0,0.000,0.819,0.181,0.6369
1900,loccitane,0611_HONEY,super hero taim love cats and songs and nature and milk tea mom of cats is my treasure pokékara the goal is pokékara super sentai series tokusatsu series domination currently the song,,98,102,2994,0,2022-06-02 14:19:02,just vote for the activities you want to support,[1クリックで未来へのアクション],9,49,0.0,0.0,0.000,0.636,0.364,0.4588
1901,loccitane,nekozoku,,tokyo nakameguro shibuya omotesando,270,99,8631,0,2022-06-02 14:18:05,just vote for the activities you want to support,[1クリックで未来へのアクション],9,49,0.0,0.0,0.000,0.636,0.364,0.4588
1902,loccitane,taejj33,lovetaec love tomorrow kairio,gotojpn,175,74,1618,0,2022-06-02 14:13:18,just vote for the activities you want to support,[1クリックで未来へのアクション],9,49,0.0,0.0,0.000,0.636,0.364,0.4588
